In [1]:
import pandas as pd
from pathlib import Path

# Define base folder
base_path = Path("..") / "Monthly Data"  # Adjust for own storage logic

# List to store individual DataFrames
dfs = []

# Folders where cleaned files will be saved
output_path = Path("cleaned_monthly_data")
output_path.mkdir(exist_ok=True)

output_path_burglary = Path("cleaned_monthly_burglary_data")
output_path_burglary.mkdir(exist_ok=True)

# Loop through each subfolder
for month_folder in base_path.iterdir():
    if month_folder.is_dir():
        for csv_file in month_folder.glob("*street*.csv"):
            try:
                # Load CSV
                df = pd.read_csv(csv_file)

                # Drop 'Context' column if it exists and is entirely empty
                if "Context" in df.columns and df["Context"].isna().all():
                    df = df.drop(columns=["Context"])

                # Drop duplicates
                df = df.drop_duplicates()

                # Drop rows with nulls in location columns
                df_cleaned = df.dropna(subset=["Longitude", "Latitude", "LSOA code"])

                # Create output filename
                month_str = month_folder.name
                
                # Save cleaned CSV
                out_file_all = output_path / f"{month_str}_cleaned.csv"
                df_cleaned.to_csv(out_file_all, index=False)
                print(f"Saved cleaned file for {month_str} to {out_file_all}")

                # Save (cleaned) only burglary CSV
                burglary_df = df_cleaned[df_cleaned["Crime type"] == "Burglary"]
                out_file_burglary = output_path_burglary / f"{month_str}_burglary_cleaned.csv"
                burglary_df.to_csv(out_file_burglary, index=False)
                print(f"Saved burglary-only file for {month_str} to {out_file_burglary}")

                dfs.append(df_cleaned)

            except Exception as e:
                print(f"Failed to process {csv_file}: {e}")

# Combine all into a single DataFrame
big_df = pd.concat(dfs, ignore_index=True)


Saved cleaned file for 2022-03 to cleaned_monthly_data\2022-03_cleaned.csv
Saved burglary-only file for 2022-03 to cleaned_monthly_burglary_data\2022-03_burglary_cleaned.csv
Saved cleaned file for 2022-03 to cleaned_monthly_data\2022-03_cleaned.csv
Saved burglary-only file for 2022-03 to cleaned_monthly_burglary_data\2022-03_burglary_cleaned.csv
Saved cleaned file for 2022-04 to cleaned_monthly_data\2022-04_cleaned.csv
Saved burglary-only file for 2022-04 to cleaned_monthly_burglary_data\2022-04_burglary_cleaned.csv
Saved cleaned file for 2022-04 to cleaned_monthly_data\2022-04_cleaned.csv
Saved burglary-only file for 2022-04 to cleaned_monthly_burglary_data\2022-04_burglary_cleaned.csv
Saved cleaned file for 2022-05 to cleaned_monthly_data\2022-05_cleaned.csv
Saved burglary-only file for 2022-05 to cleaned_monthly_burglary_data\2022-05_burglary_cleaned.csv
Saved cleaned file for 2022-05 to cleaned_monthly_data\2022-05_cleaned.csv
Saved burglary-only file for 2022-05 to cleaned_monthly

In [2]:
# Count nulls per column
null_counts = big_df.isnull().sum()

# Show columns with at least one null
print(null_counts[null_counts > 0])


Crime ID                 368743
Last outcome category    368743
dtype: int64


In [3]:
# Count nulls per column for burglaries
burg_df = big_df[big_df["Crime type"] == "Burglary"]
null_counts = burg_df.isnull().sum()

# Show columns with at least one null
print(null_counts[null_counts > 0])


Series([], dtype: int64)


In [4]:
# Create new csv files

big_df.to_csv("cleaned_crime_data.csv", index=False)
burg_df.to_csv("cleaned_burglary_data.csv", index=False)
